In [2]:
import stumpy
import matplotlib.dates as dates
from matplotlib.patches import Rectangle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import prep

In [3]:
#Bringing in the cars monthly sales data
cars = pd.read_csv('new_vehicle_by_sector.csv')

In [4]:
cars.head()

,Business,AUTO35,"2,031.3","1,834.4","1,954.5","2,164.6","2,099.3","2,092.0","2,038.2","1,976.0",...,991.9,"1,004.7","1,055.9","1,039.3",982.8,"1,214.1","1,007.4","1,022.4","1,008.3",Unnamed: 669
0,Consumer,AUTO40,"6,362.1","5,632.8","5,932.2","6,671.7","6,569.4","6,804.8","6,637.1","6,058.1",...,"1,677.7","1,641.6","1,950.3","1,816.0","1,684.7","1,769.3","1,649.7","1,650.4","1,749.0",NaN
1,NaN,NaN,1967M01,1967M02,1967M03,1967M04,1967M05,1967M06,1967M07,1967M08,...,2021M11,2021M12,2022M01,2022M02,2022M03,2022M04,2022M05,2022M06,2022M07,NaN


The columns should be the rows and the rows should be columns

In [5]:
cars.shape

(2, 670)

In [6]:
#Transposing the cars dataframe into correct layout
cars.T

,0,1
Business,Consumer,NaN
AUTO35,AUTO40,NaN
"2,031.3","6,362.1",1967M01
"1,834.4","5,632.8",1967M02
"1,954.5","5,932.2",1967M03
...,...,...
"1,214.1","1,769.3",2022M04
"1,007.4","1,649.7",2022M05
"1,022.4","1,650.4",2022M06
"1,008.3","1,749.0",2022M07


In [7]:
# Making it to be formated the correct way
cars = cars.T

In [8]:
cars.head()

,0,1
Business,Consumer,NaN
AUTO35,AUTO40,NaN
"2,031.3","6,362.1",1967M01
"1,834.4","5,632.8",1967M02
"1,954.5","5,932.2",1967M03


In [9]:
cars = cars.reset_index()

In [10]:
cars.T[0]

index        Business
0            Consumer
1                 NaN
Name: 0, dtype: object

In [11]:
cars.columns = cars.T[0].to_list()

In [12]:
cars.columns

Index(['    Business', '    Consumer', nan], dtype='object')

In [13]:
cars.head()

,Business,Consumer,NaN
0,Business,Consumer,NaN
1,AUTO35,AUTO40,NaN
2,"2,031.3","6,362.1",1967M01
3,"1,834.4","5,632.8",1967M02
4,"1,954.5","5,932.2",1967M03


In [14]:
cars = cars.drop(index=0)

In [15]:
cars = cars.drop(index=1)

In [16]:
cars.head()

,Business,Consumer,NaN
2,"2,031.3","6,362.1",1967M01
3,"1,834.4","5,632.8",1967M02
4,"1,954.5","5,932.2",1967M03
5,"2,164.6","6,671.7",1967M04
6,"2,099.3","6,569.4",1967M05


In [17]:
a = cars.columns.to_numpy()
#pandas below
#a = dftrash.columns.to_numpy()
#a[0] = 'Total production'
a[2] = 'Year'
print (a)

['    Business' '    Consumer' 'Year']


In [18]:
cars.columns

Index(['    Business', '    Consumer', 'Year'], dtype='object')

In [19]:
#Cleaning up the column titles
cars = cars.rename(columns={'    Business':'commercial', '    Consumer':'private', 'Year': 'year'})

In [20]:
cars.head()

,commercial,private,year
2,"2,031.3","6,362.1",1967M01
3,"1,834.4","5,632.8",1967M02
4,"1,954.5","5,932.2",1967M03
5,"2,164.6","6,671.7",1967M04
6,"2,099.3","6,569.4",1967M05


In [21]:
cars.year = cars.year.str.replace('M', '-')

In [22]:
cars.year = pd.to_datetime(cars.year)

In [23]:
cars.head()

,commercial,private,year
2,"2,031.3","6,362.1",1967-01-01
3,"1,834.4","5,632.8",1967-02-01
4,"1,954.5","5,932.2",1967-03-01
5,"2,164.6","6,671.7",1967-04-01
6,"2,099.3","6,569.4",1967-05-01


In [24]:
cars.dtypes

commercial            object
private               object
year          datetime64[ns]
dtype: object

In [25]:
cars['commercial'] = cars.commercial.str.replace(",","")

In [26]:
cars['private'] = cars.private.str.replace(",","")

In [27]:
cars.head()

,commercial,private,year
2,2031.3,6362.1,1967-01-01
3,1834.4,5632.8,1967-02-01
4,1954.5,5932.2,1967-03-01
5,2164.6,6671.7,1967-04-01
6,2099.3,6569.4,1967-05-01


In [28]:
cars.isnull().sum()

commercial    0
private       1
year          1
dtype: int64

In [29]:
lf_null = cars.loc[cars['year'].isnull(), ['year']]
lf_null

,year
669,NaT


In [30]:
lf_null_pri = cars.loc[cars['private'].isnull(), ['private']]
lf_null_pri

,private
669,NaN


Both of the missing values fall in the same row at the end of the data set. I will just drop the nulls

In [31]:
cars = cars.dropna()

In [32]:
cars.isnull().sum()

commercial    0
private       0
year          0
dtype: int64

In [33]:
cars.commercial.astype(float)

ValueError: could not convert string to float: '2502.2.1'

In [38]:
cars.dtypes

commercial            object
private               object
year          datetime64[ns]
dtype: object

In [52]:

cars.commercial.loc[cars.commercial.str.count('\.') > 1] = cars.commercial.loc[cars.commercial.str.count('\.') > 1].apply(lambda x: x[:-2])

In [53]:
cars[cars.commercial.str.count('\.') > 1]

,commercial,private,year


In [54]:
cars.dtypes

commercial            object
private               object
year          datetime64[ns]
dtype: object

In [55]:
cars.commercial = cars.commercial.astype(float)

In [56]:
cars.private = cars.private.astype(float)

In [57]:
cars['total_sales'] = cars.commercial + cars.private

In [58]:
cars.head()

,commercial,private,year,total_sales
2,2031.3,6362.1,1967-01-01,8393.4
3,1834.4,5632.8,1967-02-01,7467.2
4,1954.5,5932.2,1967-03-01,7886.7
5,2164.6,6671.7,1967-04-01,8836.3
6,2099.3,6569.4,1967-05-01,8668.7


In [59]:
cars.total_sales = cars.total_sales.round()

In [60]:
cars.head()

,commercial,private,year,total_sales
2,2031.3,6362.1,1967-01-01,8393.0
3,1834.4,5632.8,1967-02-01,7467.0
4,1954.5,5932.2,1967-03-01,7887.0
5,2164.6,6671.7,1967-04-01,8836.0
6,2099.3,6569.4,1967-05-01,8669.0


In [61]:
cars = cars.set_index("year").sort_index()

In [62]:
cars.dtypes

commercial     float64
private        float64
total_sales    float64
dtype: object

In [53]:
cars.to_csv('monthly_sales.csv')